This notebook creates the results of my approach in the vision room 1 easy.

The config can also be samples for e.g. hyperparameter selection.

In [13]:
import shutil
import subprocess
import time
import json
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

from evaluation_utils import (
    read_ground_truth_trajectory,
    read_ply,
    read_trajectory_dump,
    read_colord_ply,
    sample_config,
    get_cam_data,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
initial_keyframe_index_1 = 100
initial_keyframe_index_2 = 120
gt_trajectory, initial_poses = read_ground_truth_trajectory(
    "data/euroc_mav/V1_01_easy/cam0/sensor.yaml",
    "data/euroc_mav/V1_01_easy/state_groundtruth_estimate0/data.csv",
    "data/euroc_mav/V1_01_easy/cam0/data.csv",
    [initial_keyframe_index_1, initial_keyframe_index_2],
)
gt_pcl = read_ply("data/euroc_mav/V1_01_easy/pointcloud0/data.ply")

(min_x, min_y, min_z), (max_x, max_y, max_z) = np.quantile(
    gt_pcl, [0.0001, 0.9999], axis=0
)

(width, height), (fx, fy, cx, cy), (k1, k2, p1, p2) = get_cam_data(
    "data/euroc_mav/V1_01_easy/cam0/sensor.yaml"
)


config = {
    "verbose": False,
    "visual": False,
    "do_dump_map": False,
    "geometric_map": {
        "ply_path": "data/euroc_mav/V1_01_easy/pointcloud0/data.ply",
        "vertices_bounds": {
            "min_x": min_x,
            "min_y": min_y,
            "min_z": min_z,
            "max_x": max_x,
            "max_y": max_y,
            "max_z": max_z,
        },
        "voxel_size": 0.3,
    },
    "visual_odometry": {
        "images_directory": "data/euroc_mav/V1_01_easy/cam0/data",
        "images_ending": "png",
        "camera": {
            "width": width,
            "height": height,
            "parameters": {
                "fx": fx,
                "fy": fy,
                "cx": cx,
                "cy": cy,
                "k1": k1,
                "k2": k2,
                "p1": p1,
                "p2": p2,
            },
        },
        "initial_keyframe_index_1": initial_keyframe_index_1,
        "initial_keyframe_index_2": initial_keyframe_index_2,
        "initial_pose_1": {
            "qw": initial_poses[0][0][0],
            "qx": initial_poses[0][0][1],
            "qy": initial_poses[0][0][2],
            "qz": initial_poses[0][0][3],
            "x": initial_poses[0][1][0],
            "y": initial_poses[0][1][1],
            "z": initial_poses[0][1][2],
        },
        "initial_pose_2": {
            "qw": initial_poses[1][0][0],
            "qx": initial_poses[1][0][1],
            "qy": initial_poses[1][0][2],
            "qz": initial_poses[1][0][3],
            "x": initial_poses[1][1][0],
            "y": initial_poses[1][1][1],
            "z": initial_poses[1][1][2],
        },
        "initialize": {
            "keypoints": {
                "max_num": 760,
                "quality_level": 0.012,
                "min_distance_between": 9,
                "edge_margin": 20,
                "descriptor_angle_patch_radius": 15,
            },
            "matching": {
                "max_distance": 80,
                "max_second_to_first_distance_ratio": 1.18,
                "ransac": {
                    "threshold": 5e-5,
                    "min_num_inliers": 20,
                    "max_num_iterations": 100,
                },
            },
            "max_num_bundle_adjustment_iterations": 100,
        },
        "next": {
            "keypoints": {
                "max_num": 750,
                "quality_level": 0.011,
                "min_distance_between": 9,
                "edge_margin": 20,
                "descriptor_angle_patch_radius": 15,
            },
            "matching": {
                "max_image_distance": 23,
                "max_descriptor_distance": 84,
                "max_second_to_first_distance_ratio": 1.16,
                "ransac": {"max_num_iterations": 100, "image_distance_threshold": 3.6},
            },
            "keyframe": {
                "is_next": {
                    "min_distance_to_last": 0.3,
                    "max_num_landmark_keypoint_inliers": 60,
                    "min_num_landmark_keypoint_inliers": 18,
                },
                "max_num_keyframes": 29,
                "matching": {
                    "max_distance": 86,
                    "max_second_to_first_distance_ratio": 1.17,
                    "ransac": {
                        "threshold": 5e-5,
                        "min_num_inliers": 20,
                        "max_num_iterations": 100,
                    },
                    "max_epipolar_error": 3e-3,
                },
                "max_num_bundle_adjustment_iterations": 100,
            },
        },
    },
    "alignment": {
        "do_alignment": True,
        "align": {
            "distance_threshold_start": 2.0,
            "distance_threshold_end": 1.0,
            "max_num_alignment_steps": 10,
            "min_num_vertices_in_voxel": 10,
            "standard_deviation_scale": 3,
            "max_num_optimization_iterations": 20,
        },
    },
}


def run(out_path: Path, config: dict):
    if out_path.exists():
        shutil.rmtree(out_path)
    out_path.mkdir(parents=True)

    with open(out_path / "config.json", "w") as f:
        json.dump(config, f)

    print(["./cmake-build-release/evaluation", out_path, out_path / "config.json"])

    return subprocess.Popen(
        ["./cmake-build-release/evaluation", out_path, out_path / "config.json"]
    )


# Weird way to run multiprocessing but other stuff did not work for some reason.
for a in range(0, 5):
    ps = []
    for i in range(6):
        # sample_config is not doing anything here right now but it was useful to find the right hyperparameters.
        ps.append(
            run(
                Path(f"out/align_test_refined_slides/{a * 6 + i+1}"),
                sample_config(config),
            )
        )

    # Otherwise freezes will happen, idk why.
    time.sleep(120)


['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/1'), PosixPath('out/align_test_refined_slides/1/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/2'), PosixPath('out/align_test_refined_slides/2/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/3'), PosixPath('out/align_test_refined_slides/3/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/4'), PosixPath('out/align_test_refined_slides/4/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/5'), PosixPath('out/align_test_refined_slides/5/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/6'), PosixPath('out/align_test_refined_slides/6/config.json')]


[sm::SampleConsensusModel::getSamples] Can not select 8 unique points out of 7!
[sm::RandomSampleConsensus::computeModel] No samples could be selected!
[sm::SampleConsensusModel::getSamples] Can not select 8 unique points out of 5!
[sm::RandomSampleConsensus::computeModel] No samples could be selected!
W20230130 06:30:42.019855 4018872 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:30:42.028070 4018872 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:30:42.034651 4018872 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:30:42.041409 4018872 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:30:42.048389 4018872 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:30:42.051064 4018872 levenberg_marquardt_strateg

['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/7'), PosixPath('out/align_test_refined_slides/7/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/8'), PosixPath('out/align_test_refined_slides/8/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/9'), PosixPath('out/align_test_refined_slides/9/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/10'), PosixPath('out/align_test_refined_slides/10/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/11'), PosixPath('out/align_test_refined_slides/11/config.json')]
['./cmake-build-release/evaluation', PosixPath('out/align_test_refined_slides/12'), PosixPath('out/align_test_refined_slides/12/config.json')]


W20230130 06:32:09.316798 4018879 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:32:09.331349 4018879 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:32:09.340184 4018879 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:32:09.349907 4018879 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:32:09.353252 4018879 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:32:09.369563 4018879 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:32:09.403373 4018879 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
[sm::SampleConsensusModel::getSamples] Can not select 8 unique points out of 2!
[sm::RandomSampleConsensus::com

Loaded 3199068 vertices
Initialized 12180 voxels
  x: 28 (-4.50, 4.20)
  y: 29 (-3.60, 5.40)
  z: 15 (-0.30, 4.50)
Filled voxels, 464 outlier vertices
Computed 4105 local distributions


...

[sm::SampleConsensusModel::getSamples] Can not select 8 unique points out of 4!
[sm::RandomSampleConsensus::computeModel] No samples could be selected!
W20230130 06:40:05.497963 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:05.507089 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:05.510275 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:05.523036 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:05.529614 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:05.575184 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.


Loaded 3199068 vertices
Initialized 12180 voxels
  x: 28 (-4.50, 4.20)
  y: 29 (-3.60, 5.40)
  z: 15 (-0.30, 4.50)
Filled voxels, 464 outlier vertices
Computed 4105 local distributions
Loaded 3199068 vertices
Initialized 12180 voxels
  x: 28 (-4.50, 4.20)
  y: 29 (-3.60, 5.40)
  z: 15 (-0.30, 4.50)
Filled voxels, 464 outlier vertices
Computed 4105 local distributions


[sm::SampleConsensusModel::getSamples] Can not select 8 unique points out of 6!
[sm::RandomSampleConsensus::computeModel] No samples could be selected!


W20230130 06:40:10.885744 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:10.888820 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:10.900472 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:10.908911 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:10.916450 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:10.924005 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:10.926827 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute a finite step.
W20230130 06:40:10.938302 4416412 levenberg_marquardt_strategy.cc:118] Linear solver failure. Failed to compute

Loaded 3199068 vertices
Initialized 12180 voxels
  x: 28 (-4.50, 4.20)
  y: 29 (-3.60, 5.40)
  z: 15 (-0.30, 4.50)
Filled voxels, 464 outlier vertices
Computed 4105 local distributions
Loaded 3199068 vertices
Initialized 12180 voxels
  x: 28 (-4.50, 4.20)
  y: 29 (-3.60, 5.40)
  z: 15 (-0.30, 4.50)
Filled voxels, 464 outlier vertices
Computed 4105 local distributions
